In [ ]:
import pandas as pd
import altair as alt
from sublimpy import utils
alt.data_transformers.enable('json')

ls ~/Development/data/sublimationofsnow/ | grep csv

In [38]:
src = pd.read_csv(
    "~/Development/data/sublimationofsnow/monthly_planar_fits.csv", delim_whitespace=True
).groupby(['height', 'tower']).mean().sort_values('tilt').reset_index()

src[src.height.isin(['3','10'])]

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_17106/528847915.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  src = pd.read_csv(


,index,month,height,tower,a,b,c,tilt,tiltaz,W_f_1,W_f_2,W_f_3
0,28,12,10,ue,0.036421,0.000009,0.000028,0.001685,-108.294067,-0.000009,-0.000028,1.000000
1,11,11,3,ue,-0.011296,-0.000028,0.000011,0.001735,-21.282774,0.000028,-0.000011,1.000000
2,97,4,3,d,0.016353,0.000039,-0.000131,0.007854,106.688881,-0.000039,0.000131,1.000000
3,109,5,10,uw,0.003567,0.000144,-0.000041,0.008591,163.958832,-0.000144,0.000041,1.000000
4,45,1,3,ue,0.010415,-0.000007,0.000166,0.009538,-87.701836,0.000007,-0.000166,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
117,86,4,10,c,-0.072793,0.002148,0.000462,0.125869,-167.872696,-0.002148,-0.000462,0.999998
119,42,1,10,uw,0.010298,-0.001014,0.002175,0.137497,-65.014221,0.001014,-0.002175,0.999997
123,122,6,10,c,-0.073422,-0.002297,0.002184,0.181621,-43.561256,0.002297,-0.002184,0.999995
126,130,6,10,ue,0.017186,0.003017,-0.002449,0.222644,140.932007,-0.003017,0.002449,0.999992


In [ ]:
df_nopf = pd.read_parquet("tidy_df_20221101_20230619_noplanar_fit.parquet")
df_pf = pd.read_parquet("tidy_df_20221101_20230619_planar_fit.parquet")
df_pfmp = pd.read_parquet("tidy_df_20221101_20230619_planar_fit_multiplane.parquet")

df_nopf = utils.modify_df_timezone(df_nopf, 'UTC', 'US/Mountain')
df_pf = utils.modify_df_timezone(df_pf, 'UTC', 'US/Mountain')
df_pfmp = utils.modify_df_timezone(df_pfmp, 'UTC', 'US/Mountain')

df = pd.concat([
    df_nopf.query("measurement == 'w'").assign(type = "nopf"),
    df_pf.query("measurement == 'w'").assign(type = "pf"),
    df_pfmp.query("measurement == 'w'").assign(type = "pfmp"),
])

In [ ]:
line = alt.Chart().transform_calculate(y = '0').mark_rule().encode(y='y:Q')

plot = alt.Chart().mark_line().encode(
    alt.X('hoursminutes(time):T').axis(labelAlign='center'),
    alt.Y('median(value):Q').title('Vertical wind velocity (m/s)'),
    alt.Color('height:O').scale(scheme='turbo'),
    tooltip='variable',
)

alt.layer(
        line, plot, data = df.query("type == 'pfmp'").query("tower == 'c'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates').display(renderer='svg')

In [ ]:
(
    alt.layer(
        line, plot, data = df.query("type == 'nopf'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates')
    &
    alt.layer(
        line, plot, data = df.query("type == 'pf'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Slope relative local earth coordinates')
    &
    alt.layer(
        line, plot, data = df.query("type == 'pfmp'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Streamwise coordinates')
).display(renderer='svg')

In [15]:
slec_vs_streamwise_plot = (
    alt.layer(
        line, plot, data = df.query("type == 'nopf'").query("height > 1")
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates')
    &
    alt.layer(
        line, plot, data = df.query("type == 'pfmp'").query("height > 1")
    ).properties(width = 150, height = 150).facet('tower:N', title='Streamwise coordinates')
).resolve_scale(y='shared',x='shared',color='shared')
slec_vs_streamwise_plot

alt.VConcatChart(...)

In [16]:
slec_vs_streamwise_plot.save('slec_vs_streamwise_plot.png', dpi=200)

In [ ]:
start_date = '20221101'
end_date = '20230619'

def daily_w_cycle(src, title, ylim):
    return alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(value):Q').title('Wind speed (m/s)').scale(domain=ylim),
        alt.Color('height:O').scale(scheme='sinebow'),
        alt.StrokeDash('tower:N'),
        tooltip='variable',
    ).properties(width = 200, height = 200, title=title)

(

    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet"),
        'SRLEC',
        ylim=[-0.10, 0.05]
    ) | daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"),
        'Geographic',
        ylim=[-0.10, 0.05]
    ) | 
    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet"),
        'Streamwise',
        ylim=[-0.10, 0.05]
    )
).properties(title='Vertical wind speed, different coordinate systems').display(renderer='svg')